# Spaceship Adaboost Submission

In [ ]:
!pip install xgboost==2.0.3


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
!pip install lightgbm==4.3.0


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
!pip install lightgbm==4.3.0


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
!pip install catboost==1.2.5


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import StackingClassifier

# Bring the Data In

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.metrics import make_scorer, accuracy_score

# Load data
X_df = pd.read_csv('preprocessedbankchurn_train.csv')
X_df.drop("Exited", axis=1, inplace=True)
y_df = pd.read_csv('preprocessedbankchurn_train.csv')['Exited']
X_submission = pd.read_csv('preprocessedbankchurn_test.csv')

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)

# Base estimator
base_estimator = DecisionTreeClassifier(max_depth=1)

# Create pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Normalize data
    ('classifier', AdaBoostClassifier(base_estimator=base_estimator))
])


# Define GridSearch parameters
parameters = {
    'classifier__n_estimators': [200],
    'classifier__learning_rate': [1]
}



def custom_accuracy(y_true, y_prob, threshold=0.5):
    y_pred = (y_prob[:, 1] >= threshold).astype(int)
    return accuracy_score(y_true, y_pred)

# Membuat scorer kustom
accuracy_scorer = make_scorer(custom_accuracy, needs_proba=True)

# Menggunakan dalam GridSearchCV
grid_search = GridSearchCV(pipeline, parameters, scoring=accuracy_scorer, cv=5)
grid_search.fit(X_train, y_train)


# Evaluate the model
best_model = grid_search.best_estimator_
y_pred_proba = best_model.predict_proba(X_test)[:, 1]  # Get probabilities for the positive class
print("Best parameters:", grid_search.best_params_)
print("Accuracy on test set:", accuracy_score(y_test, y_pred_proba))

# Evaluation on the test set
y_pred = best_model.predict(X_test)
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Fit the model on all training data
grid_search.fit(X_df, y_df)




/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 219, in __call__
    return self._score(
  File "/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 312, in _score
    return self._sign * self._score_func(y, y_pred, **self._kwargs)
  File "/tmp/ipykernel_673/1524278816.py", line 38, in custom_accuracy
    y_pred = (y_prob[:, 1] >= threshold).astype(int)
IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

  warnings.warn(
/shared-libs/python3.9/py/lib/python3

ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline

# Load data
X_df = pd.read_csv('preprocessedbankchurn_train.csv')
X_df.drop("Exited", axis=1, inplace=True)
y_df = pd.read_csv('preprocessedbankchurn_train.csv')['Exited']
X_submission = pd.read_csv('preprocessedbankchurn_test.csv')

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)

# Define base models
base_models = [
    ('rf', RandomForestClassifier(random_state=42)),
    ('gb', GradientBoostingClassifier(random_state=42)),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)),
    ('lgbm', LGBMClassifier(random_state=42)),
    ('catboost', CatBoostClassifier(verbose=0, random_state=42)),
    ('mlp', MLPClassifier(random_state=42))
]

# Define the meta-model
meta_model = CatBoostClassifier(verbose=0, random_state=42)

# Create the voting classifier with soft voting
voting_clf = VotingClassifier(estimators=base_models, voting='soft')

# Feature selection
selector = SelectFromModel(RandomForestClassifier(n_estimators=100, random_state=42))
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

# Train the voting classifier
voting_clf.fit(X_train_selected, y_train)

# Predict and evaluate
y_pred = voting_clf.predict(X_test_selected)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy of voting ensemble: {accuracy:.4f}')

# Create the stacking ensemble with voting classifier as meta-model
stacking_clf = StackingClassifier(
    estimators=base_models,
    final_estimator=voting_clf,
    cv=5
)

# Define parameter grid for RandomizedSearchCV
param_grid = {
    'rf__n_estimators': [100],
    'rf__max_depth': [10],
    'gb__n_estimators': [100],
    'gb__learning_rate': [0.1],
    'xgb__n_estimators': [100],
    'xgb__learning_rate': [0.1],
    'lgbm__n_estimators': [100],
    'lgbm__learning_rate': [0.1],
    'catboost__iterations': [100],
    'catboost__learning_rate': [0.1],
    'mlp__hidden_layer_sizes': [(50, 50, 50)],
    'mlp__alpha': [0.01]
}

# Hyperparameter tuning with RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=stacking_clf, param_distributions=param_grid, n_iter=50, cv=3, n_jobs=-1, scoring='accuracy', random_state=42)
random_search.fit(X_train_selected, y_train)

# Predict and evaluate the best model
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test_selected)
accuracy = accuracy_score(y_test, y_pred)
print(f'Best model parameters: {random_search.best_params_}')
print(f'Accuracy of optimized stacking ensemble: {accuracy:.4f}')


[LightGBM] [Info] Number of positive: 27966, number of negative: 104061
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003316 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1856
[LightGBM] [Info] Number of data points in the train set: 132027, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.211820 -> initscore=-1.313988
[LightGBM] [Info] Start training from score -1.313988
Accuracy of voting ensemble: 0.8491
/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/model_selection/_search.py:306: UserWarning: The total space of parameters 1 is smaller than n_iter=50. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
[LightGBM] [Info] Number of positive: 18644, number of negative: 69374
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 

KernelInterrupted: Execution interrupted by the Jupyter kernel.

In [ ]:
# Predict probabilities for submission data
y_submission_proba = best_model.predict_proba(X_submission)[:, 1]  # Probabilities of Exited
print(y_submission_proba)

# Prepare submission dictionary
submission_dict = {'id': X_submission['id'], 'Exited': y_submission_proba}
submission_dict = pd.DataFrame(submission_dict)
submission_dict

[0.19618976 0.73099815 0.19885849 ... 0.20330466 0.28166607 0.2691457 ]


,id,Exited
0,165034,0.196190
1,165035,0.730998
2,165036,0.198858
3,165037,0.319627
4,165038,0.409264
...,...,...
110018,275052,0.209023
110019,275053,0.240303
110020,275054,0.203305
110021,275055,0.281666


In [ ]:
submission_dict.to_csv('banksurn_testensmbel3.csv', index=False)

In [ ]:
X_df = pd.read_csv('preprocessedbankchurn_train.csv')
X_df.drop("Exited", axis=1, inplace=True)
y_df = pd.read_csv('preprocessedbankchurn_train.csv')
y_df = y_df.loc[:, ['Exited']]
X_submission = pd.read_csv('preprocessedbankchurn_test.csv')

X_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165034 entries, 0 to 165033
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   165034 non-null  int64  
 1   CustomerId           165034 non-null  int64  
 2   CreditScore          165034 non-null  int64  
 3   Age                  165034 non-null  float64
 4   Tenure               165034 non-null  int64  
 5   Balance              165034 non-null  float64
 6   NumOfProducts        165034 non-null  int64  
 7   HasCrCard            165034 non-null  float64
 8   IsActiveMember       165034 non-null  float64
 9   EstimatedSalary      165034 non-null  float64
 10  France               165034 non-null  int64  
 11  Germany              165034 non-null  int64  
 12  Spain                165034 non-null  int64  
 13  Female               165034 non-null  int64  
 14  Male                 165034 non-null  int64  
 15  ExitedfamilySize 

In [ ]:
X_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110023 entries, 0 to 110022
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   110023 non-null  int64  
 1   CustomerId           110023 non-null  int64  
 2   CreditScore          110023 non-null  int64  
 3   Age                  110023 non-null  float64
 4   Tenure               110023 non-null  int64  
 5   Balance              110023 non-null  float64
 6   NumOfProducts        110023 non-null  int64  
 7   HasCrCard            110023 non-null  float64
 8   IsActiveMember       110023 non-null  float64
 9   EstimatedSalary      110023 non-null  float64
 10  France               110023 non-null  int64  
 11  Germany              110023 non-null  int64  
 12  Spain                110023 non-null  int64  
 13  Female               110023 non-null  int64  
 14  Male                 110023 non-null  int64  
 15  ExitedfamilySize 

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)
# Mengubah target menjadi 1D array
y_train = y_train['Exited'].values
y_test = y_test['Exited'].values


# Base estimator
base_estimator = DecisionTreeClassifier(max_depth=1)

# Membuat pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Normalisasi data
    ('classifier', AdaBoostClassifier(base_estimator=base_estimator))
])

parameters = {
    'classifier__n_estimators': [50, 100, 200],
    'classifier__learning_rate': [0.01, 0.1, 1]
}

# Menggunakan StratifiedKFold untuk handling imbalanced data
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Model Training

In [ ]:
# Membuat GridSearchCV
grid_search = GridSearchCV(pipeline, parameters, cv=cv, scoring='accuracy', verbose=1)
grid_search.fit(X_train, y_train)

#Evaluasi model
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print("Best parameters:", grid_search.best_params_)
print("Accuracy on test set:", accuracy_score(y_test, y_pred))

Fitting 10 folds for each of 9 candidates, totalling 90 fits


KeyboardInterrupt: 

In [ ]:
# Evaluasi pada validation set
y_pred = grid_search.predict(X_test)
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.95      0.92     26052
           1       0.75      0.56      0.64      6955

    accuracy                           0.87     33007
   macro avg       0.82      0.76      0.78     33007
weighted avg       0.86      0.87      0.86     33007

Confusion Matrix:
[[24755  1297]
 [ 3028  3927]]


In [ ]:
best_ada = grid_search.best_estimator_
y_pred_light = best_ada.predict(X_test)
print("Best parameters:", grid_search.best_params_)
print("Accuracy:", accuracy_score(y_test, y_pred_light))
print("Classification Report:\n", classification_report(y_test, y_pred_light))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_light))

Best parameters: {'classifier__learning_rate': 1, 'classifier__n_estimators': 200}
Accuracy: 0.8689671887781379
Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.95      0.92     26052
           1       0.75      0.56      0.64      6955

    accuracy                           0.87     33007
   macro avg       0.82      0.76      0.78     33007
weighted avg       0.86      0.87      0.86     33007

Confusion Matrix:
 [[24755  1297]
 [ 3028  3927]]


# Submission Prediction

In [ ]:
X_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110023 entries, 0 to 110022
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   110023 non-null  int64  
 1   CustomerId           110023 non-null  int64  
 2   CreditScore          110023 non-null  int64  
 3   Age                  110023 non-null  float64
 4   Tenure               110023 non-null  int64  
 5   Balance              110023 non-null  float64
 6   NumOfProducts        110023 non-null  int64  
 7   HasCrCard            110023 non-null  float64
 8   IsActiveMember       110023 non-null  float64
 9   EstimatedSalary      110023 non-null  float64
 10  France               110023 non-null  int64  
 11  Germany              110023 non-null  int64  
 12  Spain                110023 non-null  int64  
 13  Female               110023 non-null  int64  
 14  Male                 110023 non-null  int64  
 15  ExitedfamilySize 

In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 132027 entries, 149380 to 121958
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   132027 non-null  int64  
 1   CustomerId           132027 non-null  int64  
 2   CreditScore          132027 non-null  int64  
 3   Age                  132027 non-null  float64
 4   Tenure               132027 non-null  int64  
 5   Balance              132027 non-null  float64
 6   NumOfProducts        132027 non-null  int64  
 7   HasCrCard            132027 non-null  float64
 8   IsActiveMember       132027 non-null  float64
 9   EstimatedSalary      132027 non-null  float64
 10  France               132027 non-null  int64  
 11  Germany              132027 non-null  int64  
 12  Spain                132027 non-null  int64  
 13  Female               132027 non-null  int64  
 14  Male                 132027 non-null  int64  
 15  ExitedfamilySize 

In [ ]:
# For training, we use ALL data from spaceship_train_X_v2.csv and spaceship_train_y.csv
grid_search.fit(X_df, y_df['Exited'].values)
# Prediksi data submission
y_prediction = grid_search.predict(X_submission)
print(y_prediction)


Fitting 10 folds for each of 9 candidates, totalling 90 fits
[0 1 0 ... 0 0 0]


In [ ]:
X_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110023 entries, 0 to 110022
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   110023 non-null  int64  
 1   CustomerId           110023 non-null  int64  
 2   CreditScore          110023 non-null  int64  
 3   Age                  110023 non-null  float64
 4   Tenure               110023 non-null  int64  
 5   Balance              110023 non-null  float64
 6   NumOfProducts        110023 non-null  int64  
 7   HasCrCard            110023 non-null  float64
 8   IsActiveMember       110023 non-null  float64
 9   EstimatedSalary      110023 non-null  float64
 10  France               110023 non-null  int64  
 11  Germany              110023 non-null  int64  
 12  Spain                110023 non-null  int64  
 13  Female               110023 non-null  int64  
 14  Male                 110023 non-null  int64  
 15  ExitedfamilySize 

In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 132027 entries, 149380 to 121958
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   132027 non-null  int64  
 1   CustomerId           132027 non-null  int64  
 2   CreditScore          132027 non-null  int64  
 3   Age                  132027 non-null  float64
 4   Tenure               132027 non-null  int64  
 5   Balance              132027 non-null  float64
 6   NumOfProducts        132027 non-null  int64  
 7   HasCrCard            132027 non-null  float64
 8   IsActiveMember       132027 non-null  float64
 9   EstimatedSalary      132027 non-null  float64
 10  France               132027 non-null  int64  
 11  Germany              132027 non-null  int64  
 12  Spain                132027 non-null  int64  
 13  Female               132027 non-null  int64  
 14  Male                 132027 non-null  int64  
 15  ExitedfamilySize 

In [ ]:
submission_dict = {'id':X_submission['id'], 'Exited':y_prediction}
submission_dict = pd.DataFrame(submission_dict)
submission_dict

,id,Exited
0,165034,0
1,165035,1
2,165036,0
3,165037,0
4,165038,0
...,...,...
110018,275052,0
110019,275053,0
110020,275054,0
110021,275055,0


## Export CSV

In [ ]:
submission_dict.to_csv('bankChurn_adaboost3.csv', index=False)

AttributeError: 'dict' object has no attribute 'to_csv'

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2a79941c-6614-47fe-9427-0e9f23998893' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>